In [8]:
import os, sys, matplotlib, pylab
import numpy  as np
from astropy.io import fits
from matplotlib import pyplot as plt
import matplotlib
import re
from astropy.table import Table, Column
from astropy.io import ascii
import scipy.stats as ss

def norm(a):
    p = (a-np.median(a))/np.median(a)
    return p
plt.switch_backend('Qt5Agg')

from astropy.stats import LombScargle
import emcee
import corner


    

#Working with EPIC sorted values, except mentioned otherwise

In [9]:
#Analyzing flares
catfile = '/home/vatsal/UThesis/K2/K2_Refined/SC/sc-md-rotflag.cat'
tab = Table.read(catfile,format='ascii.tab')
rdblist = ['EPIC_'+str(i)+'_mast.fits.rdb' for i in tab['EPIC']]
path_to_ktwodetdir = '/home/vatsal/UThesis/K2/K2_Refined/SC/ktwo-detrended'
infolist,energylist,allflareamp,durlist = flaredetector(path_to_ktwodetdir,rdblist,catfile)

# tup = (re.findall('\d+', s)[0],timeint,len(flareposition),peak_A,peak_E,P_rot,VP)

EPIC_206019387_mast.fits.rdb
EPIC_206050032_mast.fits.rdb
EPIC_206135809_mast.fits.rdb
EPIC_206169988_mast.fits.rdb
EPIC_206262336_mast.fits.rdb
EPIC_210317378_mast.fits.rdb
EPIC_210489654_mast.fits.rdb
EPIC_210499476_mast.fits.rdb
EPIC_210758829_mast.fits.rdb
EPIC_210894955_mast.fits.rdb
EPIC_210942999_mast.fits.rdb
EPIC_211046195_mast.fits.rdb
EPIC_211069418_mast.fits.rdb
EPIC_211081477_mast.fits.rdb
EPIC_211082433_mast.fits.rdb
EPIC_211112686_mast.fits.rdb
EPIC_211129272_mast.fits.rdb
EPIC_211400847_mast.fits.rdb
EPIC_211480655_mast.fits.rdb
EPIC_211892034_mast.fits.rdb
EPIC_211906940_mast.fits.rdb
EPIC_211945363_mast.fits.rdb
EPIC_212029094_mast.fits.rdb
EPIC_212104761_mast.fits.rdb
EPIC_212144340_mast.fits.rdb
EPIC_212178513_mast.fits.rdb
EPIC_212285603_mast.fits.rdb
EPIC_212518629_mast.fits.rdb
EPIC_212692562_mast.fits.rdb
EPIC_212820594_mast.fits.rdb


In [27]:
# Plotting flares
# flaredetector(path_to_ktwodetdir,rdblist[0:2],catfile)

#Counting flares
print [max(i) for i in allflareamp]

[1.0267511308220132, 9.4562501124523539, 36.860940343254498, 8.0152261583685842, 16.698155607003653, 3.4171801957474477, 0.84577683211756027, 8.3771927079207913, 51.237456821541095, 5.1661022957644791, 1.9599406683881673, 21.424279728015783, 12.020551945193606, 1.5961496994191386, 10.672403942298047, 12.599198103024992, 5.7645902852532478, 2.752019412752658, 0.18772405110369228, 0.11136957855027742, 28.287459967180745, 0.2646937143980943, 0.47333858801512291, 1.5279327350642233, 0.37002756637979772, 12.63563994372994, 0.10081451625961488, 0.49152200127711487, 1.1256949938783132, 142.696101866421]


In [6]:
#Histogram for number distribution of flares

hist, bins = np.histogram([infolist[i][2] for i in range(len(infolist))], bins=50)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.xlabel('Number of flares')
plt.ylabel('Number of M dwarfs')
plt.show()

# print [infolist[i][2] for i in range(len(infolist))]

In [16]:
print durlist

[[0.10216144454170717, 0.1021615193967591, 0.12259368468949106, 0.18389000514434883, 0.10216053952171933, 0.081728060948080383, 0.08172750076482771, 0.10215924708609236, 0.14302274915098678, 0.10215881448675646, 0.12259066790284123, 0.081726730670197867, 0.081726401214837097, 0.10215792122471612, 0.16345265889685834, 0.1021578066793154, 0.10215787489141803, 0.1021578421787126, 0.12258924286288675, 0.10215768444322748, 0.081726115997298621, 0.12258916917198803, 0.081726209245971404, 0.10215765664179344, 0.10215778811107157, 0.12258939728781115], [0.102161183647695, 0.10216122171550523, 0.10216095745272469, 0.10216085593856405, 0.081728848737839144, 0.081728635581384879, 0.081728692275646608, 0.10216033149481518, 0.081727822071115952, 0.081727420671086293, 0.12259081016236451, 0.081726982360123657, 0.10215765859175008, 0.12258924263005611, 0.14302073317958275, 0.14302064511866774, 0.10215763201995287, 0.10215769686328713, 0.081726346710638609, 0.10215772814990487, 0.10215798606805038], [

In [49]:

#Create colormap according to the temperature values(for CFFD)
tlist = []
for myList in energylist:
    for item in myList:
        tlist.append(item[1])
norm = matplotlib.colors.Normalize(
    vmin=np.min(tlist),
    vmax=np.max(tlist))
# choose a colormap
c_m = matplotlib.cm.jet

# create a ScalarMappable and initialize a data structure
s_m = matplotlib.cm.ScalarMappable(cmap=c_m, norm=norm)
s_m.set_array([])

##Cumulative FFD 
# erglist = []
# for myList in energylist:
#     for item in myList:
#         erglist.append(item)

for s in range(len(energylist)-29):
    
    erglist = [energylist[s][k][0] for k in range(len(energylist[s]))]
    tefflist = [energylist[s][k][1] for k in range(len(energylist[s]))]


    #Removing the useless column name tags with each element in the erglist
    xnum2 = []
    for myList in np.array(erglist):
        for item in myList:
            xnum2.append(item)
#     print xnum2
    erglist=xnum2
    xnum2 = []
    for myList in np.array(tefflist):
        for item in myList:
            xnum2.append(item)
    tefflist=xnum2
    



    # fit a power law by Maximum Likelihood
    
    sorted_erglist = sorted(erglist)
    N=len(sorted_erglist)
    F = np.array(range(N))
    

    
    
    sorted_erglist = [10**i for i in sorted_erglist]                           #serglist is now actual energy values(and not their log)
    
    
    serglist = sorted_erglist[:]
    summation = 0.
    
    


    ##Calcultating alpha
    
    slist = serglist[12:]
    for i,k in enumerate(slist):
        summation = summation + np.log(k/min(slist)) 
        
    alpha = (1+len(slist)/(summation))
    error = ((1+len(slist))**0.5)/(summation)

    print 'alpha = %s'%alpha
    print error

    c = (alpha-1)*((min(slist))**(alpha-1))
    slope = -alpha+1
    print slope
    print c
    
    
    # Calculate the fit
    energy1 = np.linspace(min(serglist),max(serglist[:-1]),1000)
    fit = (slope)*np.log10(energy1) - np.log10(infolist[0][1]) + np.log10(c) + 0.8
    plt.plot(np.log10(sorted_erglist[:-1]),np.log10(F[:0:-1]/infolist[0][1]),'k.')
    plt.plot(np.log10(energy1),fit,'r--')
    plt.xlabel('log Flare energy(ergs)')
    plt.ylabel('Cumulative $logN_{flares}$/day')
    plt.title('Cumulative Flare Frequency Distribution for EPIC{} \n Power law index={}$\pm${} \n Slope={}'.format(infolist[s][0],alpha,error,-alpha+1))
    plt.show()
    
    
#     weights = np.exp(1/(np.log10(np.array(range(len(serglist)))[:0:-1]))**2)
    

#     
#     plt.legend(loc='lower left')




alpha = 2.92163393001
0.53160401489
-1.92163393001
1.99126455933e+62


In [ ]:
    m,inte = np.polyfit(np.log10(serglist[:-1]),np.log10(np.array(range(len(serglist)))[:0:-1]),1,w=weights)
    print m
    print inte
    energy = np.linspace(min(serglist[:-1]),max(serglist[:-1]),1000)
    plt.plot(np.log10(serglist),np.log10(F[::-1]/infolist[s][1]),'.',color=s_m.to_rgba(tefflist[0]))
    plt.plot(np.log10(energy),m*np.log10(energy)+inte-np.log10(infolist[s][1]),color=s_m.to_rgba(tefflist[0]),label='(LS)Slope =%s'%m)
#     plt.plot(np.log10(energy1),fit,'r',label='(MLE)Slope =%s'%slope)
    plt.xlabel('log Flare energy(ergs)')
#     plt.title('Cumulative FFD')
    plt.title('Cumulative FFD for {} \n Power law index={} \n Slope={}'.format(infolist[s][0],-m+1,m))
    plt.ylabel('Cumulative log N_flares/day')
    plt.colorbar(s_m)
    plt.show()

In [31]:
#Rotation vs Activity

#tup = (re.findall('\d+', s)[0],timeint,len(flareposition),peak_A,peak_E,Teff,P_rot,VP)  -> the elements in infolist

ind = [i for i,k in enumerate(infolist) if k[7]=='Y']
ilist = [infolist[i] for i in ind]
famplist = [allflareamp[i] for i in ind]
fenlist = [energylist[i] for i in ind]


#Create proper lists for analysis

rotampteff = []
for x,i in enumerate(famplist):        
    for k in i:
        if(k!=0):
            rotampteff.append((k,ilist[x][6],ilist[x][5],max(i),min(i)))

rotenteff = []
for x,i in enumerate(fenlist):        
    for k in i:
        if(k[0]!=0):
            rotenteff.append((k[0],ilist[x][6],ilist[x][5],max([l[0] for l in i]),min([l[0] for l in i])))

#Create minmaxprot lists
            
# mmamprot = []
# for x,i in enumerate(famplist):
#     mmamprot.append((max(i),min(i),ilist[x][6],ilist[x][5]))

# mmenprot = []
# for x,i in enumerate(fenlist):
#     mmenprot.append((max([k[0] for k in i]),min([k[0] for k in i]),ilist[x][6],ilist[x][5]))


            
#Create colormap for temperature range
norm = matplotlib.colors.Normalize(
    vmin=np.min([i[2] for i in rotampteff]),
    vmax=np.max([i[2] for i in rotampteff]))
# choose a colormap
c_m = matplotlib.cm.jet

# create a ScalarMappable and initialize a data structure
s_m = matplotlib.cm.ScalarMappable(cmap=c_m, norm=norm)
s_m.set_array([])

#func to remove useless column tags
def ttolist(l):
    xnum2 = []
    for myList in np.array(l):
        for item in myList:
            xnum2.append(item)
    return xnum2
    


amp, rot, teff , maxamp, minamp = map(list,zip(*rotampteff))
en, rot, teff2 , maxen, minen = map(list,zip(*rotenteff))

rot = ttolist(rot)
teff = ttolist(teff)
en = ttolist(en)
maxen = ttolist(maxen)
minen = ttolist(minen)

print maxen
print minen



#Plot!!
#rot vs amp
label_size = 15
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size 


for i in range(len(teff)):
    plt.plot(np.log10(rot[i]),np.log10(amp[i]),'o',color=s_m.to_rgba(teff[i]))
    plt.plot(np.log10(rot[i]),np.log10(maxamp[i]),'^',ms=10,color=s_m.to_rgba(teff[i]))
    plt.plot(np.log10(rot[i]),np.log10(minamp[i]),'v',ms=10,color=s_m.to_rgba(teff[i]))

plt.xlabel('$log_{10}(P_{rot}[d])$',fontsize=15)
plt.ylabel('$log_{10}(Amplitude(\Delta F/F)[\%])$',fontsize=15)
cbar = plt.colorbar(s_m)
cbar.set_label('$T_{eff}$',size=18)
plt.show()


# ##rot vs energy plot
# for i in range(len(teff)):
#     plt.plot(np.log10(rot[i]),en[i],'o',color=s_m.to_rgba(teff[i]))
#     plt.plot(np.log10(rot[i]),maxen[i],'^',ms=10,color=s_m.to_rgba(teff[i]))
#     plt.plot(np.log10(rot[i]),minen[i],'v',ms=10,color=s_m.to_rgba(teff[i]))
# plt.xlabel('$log_{10}(P_{rot}[d])$',fontsize=15)
# plt.ylabel('$log_{10}(E[ergs])$',fontsize=15)

# cbar = plt.colorbar(s_m)
# cbar.set_label('$T_{eff}$',size=18)
# plt.show()


# ##rot vs number of flares per day

# #tup = (re.findall('\d+', s)[0],timeint,len(flareposition),peak_A,peak_E,Teff,P_rot,VP)  -> the elements in infolist

# temp = ttolist([i[5] for i in ilist])

# for i in range(len(ilist)):
#     plt.plot(np.log10(ilist[i][6]),ilist[i][2]/ilist[i][1],'o',color=s_m.to_rgba(temp[i]))
# plt.xlabel('$log_{10}(P_{rot}[d])$',fontsize=15)
# plt.ylabel('$N_{flares}/day$',fontsize=15)
# cbar = plt.colorbar(s_m)
# cbar.set_label('$T_{eff}$',size=18)
# plt.show()



[32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.885462981016481, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 32.75387011686621, 33.011406470160125, 33.011406470160125, 33.011406470160125, 33.011406470160125, 

In [12]:
for i in range(len(ilist)):
    print ilist[i][5]

Teff
----
3472
Teff
----
2617
Teff
----
2418
Teff
----
3495
Teff
----
3788
Teff
----
3751
Teff
----
3325
Teff
----
3027
Teff
----
3550
Teff
----
3696
Teff
----
2432
Teff
----
3675
Teff
----
3567
Teff
----
3700
Teff
----
3729
Teff
----
3641
Teff
----
3685


In [ ]:
#emcee fit

sort_erglist = sorted_erglist[10:]
xdata = np.log10(sort_erglist[:-1])
ydata = np.log10(np.array(range(len(sort_erglist[:])))[:0:-1]/infolist[0][1])
yerr = np.zeros(len(np.log10(sort_erglist[:-1])))   


def lnlike(theta, x, y, yerr):
    m, b, lnf = theta
    model = m*x + b
    inv_sigma2 = 1.0/(yerr**2 + model**2*np.exp(2*lnf))
    return -0.5*(np.sum((y-model)**2*inv_sigma2 - np.log(inv_sigma2)))

import scipy.optimize as op
nll = lambda *args: -lnlike(*args)
result = op.minimize(nll,[-1.40879110678,0,0],args=(xdata, ydata, yerr),method='Nelder-Mead')
m_ml, b_ml, lnf_ml = result["x"]

def lnprior(theta):
    m, b, lnf = theta
    if -5.0 < m < 0.5 and 0.0 < b < 10.0 and -10.0 < lnf < 1.0:
        return 0.0
    return -np.inf

def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, x, y, yerr)

ndim, nwalkers = 3, 100
pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]

#Set and run the sampler
import emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(xdata, ydata, yerr))
sampler.run_mcmc(pos, 1000)

samples = sampler.chain[:,:,:].reshape((-1, ndim))

#corner plot
import corner
fig = corner.corner(samples, labels=["$m$", "$b$", "$\ln\,f$"],
                       quantiles=[0.16, 0.5, 0.84],show_titles=True, title_kwargs={"fontsize": 12})
plt.show()

samples[:, 2] = np.exp(samples[:, 2])
m_mcmc, b_mcmc, f_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples, [16, 50, 84],
                                                axis=0)))

xl = np.linspace(min(xdata),max(xdata),100)
for m, b, lnf in samples[np.random.randint(len(samples), size=100)]:
    plt.plot(xl, m*xl + b, color="k", alpha=0.1)
plt.plot(xl, xl*m_mcmc[0] + b_mcmc[0], color="r", lw=2, alpha=0.8)
plt.plot(xdata, ydata,"ok")
plt.show()

In [26]:
xdata = np.log10(sort_erglist[:-1])
ydata = np.array(range(len(sort_erglist[:])))[:0:-1]
print xdata
print ydata

[ 31.87444822  31.92999654  31.9661969   32.00657102  32.0066226
  32.02723655  32.0956428   32.13919575  32.17088687  32.20249506
  32.22260489  32.2277679   32.27225401  32.30411946  32.3210658
  32.34607729  32.38772278  32.41895619  32.44072991  32.44256203
  32.47947028  32.53282955  32.57415871  32.70324012  32.86694503]
[25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1]


In [4]:
#First create rdblist
#Also provide the path to catfile with rotflag

##First test with one file

def flaredetector(path_to_ktwodetdir,rdblist,catfile):
    binsize = 4

    infolist = []            #to store duration of observation(in days), peak flare amp and energy, and number of flares

    energylist = []

    allflareamp = []

    durlist = []
    
    for s in rdblist:
        
    

        f = open(path_to_ktwodetdir+'/'+s, 'r')  

        d = f.read()
        table = ascii.read(d)
        time_det = np.float64(table['jdb'])
        flux_det = np.float64(table['flux'])
        trend_t = np.float64(table['trendt'])
        trend_p = np.float64(table['trendp'])

        timeint = max(time_det)-min(time_det)

        f.close()


        #mask the nan point
        # masknan = -np.isnan(data['flux'])
        # time = data['time'][masknan]
        # flux = data['flux'][masknan]/np.mean(data['flux'][masknan])
        # flux_raw = data['flux'][masknan]/np.mean(data['flux'][masknan])


        mnanm = ~np.isnan(flux_det)
        time = time_det[mnanm]
        flux_d = 100.*norm(flux_det[mnanm])-100.*norm(trend_p[mnanm])-100.*norm(trend_t[mnanm])
        flux_r = 100.*norm(flux_det[mnanm])-100.*norm(trend_p[mnanm])-100.*norm(trend_t[mnanm])

        #make empty array as same length as masked flux
        fluxC = np.zeros(len(flux_d))


        #separate gap when time step > 3*time step 
        separationtime = []
        separationtime.append(0)
        for i in range(len(time)-1):
            if time[i+1] - time[i] > 3*(time[2] - time[1]):
                separationtime.append(i)
        separationtime.append(len(time))

        #print separationtime

        #do j times
        for j in range(2):
            for k in range(len(separationtime)-1):
                #take 10 point median and replace it
                for i in range(separationtime[k],separationtime[k+1]):
                    if i >= (separationtime[k]+binsize) and i <= (separationtime[k+1]-binsize): 
                        fluxC[i] = np.median(flux_d[i-binsize:i+binsize])
                    else :
                        fluxC[i] = flux_d[i]


                #Observed subtract Caculate 

                fluxdiff = flux_d-fluxC

                #move out the peak value > 1 sigma
                maskpeak = fluxdiff > 1.0*fluxdiff.std()

                #if the peak larger than standard deviation then replace the number by 10 point median
                for n, i in enumerate(maskpeak):
                    if i:
                        if n >= (separationtime[k]+binsize) and n <= (separationtime[k+1]-binsize):
                            flux_d[n] = np.median(flux_d[n-binsize:n+binsize])
                        else:
                            flux_d[n] = flux_d[n]


        #Plotting the O-C flux

        OminusC = flux_r - fluxC

        #plt.plot(time,flux_raw,'o-')
        #plt.plot(time,fluxC,'o-')
        #     plt.plot(time,OminusC,'.-')
        #     plt.show()

        flare = np.where(OminusC > 3.0*OminusC.std())

        #############

        OminusC = flux_r-fluxC

        flares = np.zeros(len(OminusC))

        flareselection = OminusC > 3.0*OminusC.std()
        #print 3.0*OminusC.std()
        #print 3.0*fluxC.std()

        for n, i in enumerate(flareselection):
            if i:
                if OminusC[n-2] > 0.05*OminusC[n]:
                    flares[n-2] = OminusC[n-2]

                if OminusC[n-1] > 0.05*OminusC[n]:
                    flares[n-1] = OminusC[n-1]

                flares[n] = OminusC[n]

                if OminusC[n] > OminusC[n+1] or OminusC[n+1] > 0.05*OminusC[n]:
                    flares[n+1] = OminusC[n+1]

                if OminusC[n+1] > OminusC[n+2] or OminusC[n+2] > 0.05*OminusC[n]:
                    flares[n+2] = OminusC[n+2]

                if OminusC[n+2] > OminusC[n+3] or OminusC[n+3] > 0.05*OminusC[n]:
                    flares[n+3] = OminusC[n+3]

                if n+4 < len(flareselection):
                    if OminusC[n+3] > OminusC[n+4] or OminusC[n+4] > 0.05*OminusC[n]:
                        flares[n+4] = OminusC[n+4]

                if n+5 < len(flareselection):
                    if OminusC[n+4] > OminusC[n+5] or OminusC[n+5] > 0.05*OminusC[n]:
                        flares[n+5] = OminusC[n+5]

        #change all minus value to 0

        for i in range(len(flares)):
            if flares[i] < 0:
                flares[i] = 0

        #remove fake flares

        for i in range(len(flares)-1):        
            if  flares[i] < 3.0*fluxC.std() and \
                flares[i-1] == 0 and \
                flares[i+1] == 0 :
                    flares[i] = 0   
            elif \
                flares[i] < 3.0*fluxC.std() and \
                flares[i-2] == 0 and \
                flares[i+1] == 0 :
                    flares[i] = 0
            elif \
                flares[i] < 3.0*fluxC.std() and \
                flares[i-1] == 0 and \
                flares[i+2] == 0 :
                    flares[i] = 0
        for i in range(len(flares)-1):        
            if  flares[i-1] == 0 and \
                flares[i+1] == 0 :
                    flares[i] = 0
        
        
        #Reading the stellar properties for flare energy calculation
        tab = Table.read(catfile,format='ascii.tab')
        z = [i for i,k in enumerate(tab['EPIC']) if str(k)==re.findall('\d+', s)[0]]
        R = tab['Radius'][z]
        Rsun = 6.96e08
        sigma = 5.67e-8         
        Teff = tab['Teff'][z]
        L = 4*np.pi*((R*Rsun)**2)*sigma*(Teff**4)*(10**7)
        

        #counting flares and find their position
        flarecount = []
        flareposition = []
        flareamp = []

        def findlocalmax(queue):
            maximum = 0
            for i in range(len(queue)):
                if (queue[i] > maximum):
                    maximum = queue[i]
                    max_position = len(queue) - i
            return maximum, max_position

        is_zero = 1
        queue = []
        dur=[]

        for i in range(len(flares)):                              #second if execs when first flare point is encountered,switches is_zero to 0
            if ((not is_zero) and flares[i] == 0):                #the whole flare is queued till the last point(when it drops to zero)
                is_zero = 1                                       #then the first if execs, does the analysis for the whole flare,
                t2 = time[i]                                                  #empties the queue, and switches is_zero to 1, goes to next point(queue.append)
                #print (findlocalmax(queue))                     #executed only in the next iteration
                #find flareposition
                index = i - findlocalmax(queue)[1]
                flareposition.append(index)
                flareamp.append(findlocalmax(queue)[0])

                #sum the each point of the data in one flare
                total = sum(queue)
                flarecount.append((np.log10(total*30*60*L/100.),Teff))      #each point is 30 minutes long
                duration = t2-t1
                dur.append(duration)
                queue = []
                continue
            if (is_zero and flares[i] != 0):
                is_zero = 0
                t1=time[i-1]
            queue.append(flares[i])

        #print flareposition
        #print flarecount
        #print L
        #print len(flareposition)

        energylist.append(flarecount)
        allflareamp.append(flareamp)
        durlist.append(dur)
        
        if(len(flarecount)!=0):
            peak_E = max(flarecount)
            peak_A = max(flares)
        else:
            peak_E = 0.
            peak_A = 0.
        
        VP = tab['VP'][z]
        P_rot = tab['P_rot'][z]
        tup = (re.findall('\d+', s)[0],timeint,len(flareposition),peak_A,peak_E,Teff,P_rot,VP)
        infolist.append(tup)
        print s

#         plt.plot(time,flares,'k.-')
#         plt.xlabel('Time[BJD-2454833]')
#         plt.ylabel(r'$\Delta F/F[\%]$')
#         plt.title('EPIC'+re.findall('\d+', s)[0])
#         plt.show()

        #     plt.plot(time,100.*norm(flux_det[mnanm])-20,'b.-')
        #     plt.plot(time,100.*norm(flux_det[mnanm])-100.*norm(trend_p[mnanm]),'k.-')

        #     plt.plot(time,flux_d+20,'g.-')


        #     plt.show()

    return infolist,energylist,allflareamp,durlist
